In [1]:
import timm
import torch
import timm.data
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torchvision.models as models
import torchvision.transforms as T
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

cols = ['ImageFilePath', 'ScientificName']

df = canon_df[cols]


In [3]:
df.head(2)


,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus


In [4]:

model = timm.create_model('efficientnet_b3a', pretrained=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)


for param in model.parameters():
    param.requires_grad = False


preprocess_train = timm.data.create_transform(
    input_size=(224, 224),
    is_training=True,
)

preprocess_val = timm.data.create_transform(
    input_size=(224, 224),
    is_training=False,
)


transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name efficientnet_b3a to current efficientnet_b3.
  model = create_fn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model.forward_features(image)
    
    return features.cpu().numpy().flatten() 


In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df['ImageFilePath'])])

le = LabelEncoder()

y = le.fit_transform(df['ScientificName'])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 1622/1622 [01:40<00:00, 16.20it/s]


In [7]:
models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),   
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=42)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 65.23% | Prec: 56.60% | Rec: 65.23% | F1: 58.59% | Bal Acc: 15.16% | MCC: 0.4220


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression        | Acc: 78.46% | Prec: 75.87% | Rec: 78.46% | F1: 76.93% | Bal Acc: 32.93% | MCC: 0.6689


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVMLinear                 | Acc: 80.92% | Prec: 72.37% | Rec: 80.92% | F1: 74.92% | Bal Acc: 20.89% | MCC: 0.6999


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 44.62% | Prec: 50.96% | Rec: 44.62% | F1: 36.45% | Bal Acc: 11.78% | MCC: 0.0933
MLP_Baseline              | Acc: 80.00% | Prec: 75.45% | Rec: 80.00% | F1: 77.00% | Bal Acc: 28.06% | MCC: 0.6848


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_NearestNeighbor,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax discedens
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus


In [9]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.652308,0.565997,0.652308,0.585866,0.151577,0.421974
LogisticRegression,LogisticRegression,0.784615,0.758707,0.784615,0.769314,0.329262,0.668938
SVMLinear,SVMLinear,0.809231,0.723683,0.809231,0.749192,0.208899,0.699885
NearestNeighbor,NearestNeighbor,0.446154,0.509641,0.446154,0.364512,0.117777,0.093336
MLP_Baseline,MLP_Baseline,0.8,0.754458,0.8,0.770035,0.280576,0.684781


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/49.EfficientNet-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/49.EfficientNet-linear-probing-species-metrics.csv", index=False)